In [75]:
import os
from pathlib import Path
import re
import numpy as np
from itertools import combinations
from collections import Counter
from sympy import Interval, Union

In [92]:
# Preprocessing
raw_rules = raw_rules.split('\n')
my_ticket = np.array(my_ticket.split('\n')[1].split(','))
tickets = np.array([ ticket.split(',') for ticket in tickets.split('\n')[1:-1] ])

In [93]:
rules = {}

for line in raw_rules:
    name, values = line.split(': ')
    rules[name] = []
    
    for bounds in values.split(' or '):
        lower, higher = map(lambda s: int(s), bounds.split('-'))
        rules[name] = [ *rules[name], (lower, higher + 1) ]

In [94]:
values = []

for val in rules.values():
    values = [ *val, *values ]

[(30, 630),
 (637, 969),
 (32, 817),
 (825, 972),
 (44, 241),
 (265, 961),
 (45, 295),
 (317, 969),
 (35, 446),
 (469, 955),
 (35, 283),
 (288, 965),
 (36, 891),
 (901, 952),
 (35, 693),
 (706, 958),
 (38, 122),
 (129, 959),
 (32, 807),
 (815, 960),
 (25, 321),
 (344, 955),
 (40, 676),
 (685, 950),
 (44, 739),
 (762, 971),
 (32, 561),
 (567, 952),
 (38, 267),
 (274, 956),
 (41, 482),
 (492, 954),
 (48, 213),
 (233, 951),
 (36, 375),
 (394, 959),
 (45, 102),
 (118, 968),
 (42, 571),
 (579, 961)]

In [95]:
def union(data):
    """ Union of a list of intervals e.g. [(1,2),(3,4)] """
    intervals = [Interval(begin, end) for (begin, end) in data]
    u = Union(*intervals)
    return [list(u.args[:2])] if isinstance(u, Interval) \
       else list(u.args)

large_range = range(*tuple(union(values)[0]))
large_range

range(25, 972)

In [134]:
errors = []
valid_ticket_indices = []
for i, ticket in enumerate(tickets):
    ticket_errors = [ int(value) for value in ticket if int(value) not in large_range ]
    if len(ticket_errors) > 0:
        errors = [ *errors, *ticket_errors ]
    valid_ticket_indices.append(len(ticket_errors) == 0)

In [131]:
np.sum(errors)

24980

In [137]:
rules

{'departure location': [(42, 571), (579, 961)],
 'departure station': [(45, 102), (118, 968)],
 'departure platform': [(36, 375), (394, 959)],
 'departure track': [(48, 213), (233, 951)],
 'departure date': [(41, 482), (492, 954)],
 'departure time': [(38, 267), (274, 956)],
 'arrival location': [(32, 561), (567, 952)],
 'arrival station': [(44, 739), (762, 971)],
 'arrival platform': [(40, 676), (685, 950)],
 'arrival track': [(25, 321), (344, 955)],
 'class': [(32, 807), (815, 960)],
 'duration': [(38, 122), (129, 959)],
 'price': [(35, 693), (706, 958)],
 'route': [(36, 891), (901, 952)],
 'row': [(35, 283), (288, 965)],
 'seat': [(35, 446), (469, 955)],
 'train': [(45, 295), (317, 969)],
 'type': [(44, 241), (265, 961)],
 'wagon': [(32, 817), (825, 972)],
 'zone': [(30, 630), (637, 969)]}

In [143]:
valid_tickets = tickets[valid_ticket_indices]
m, n = valid_tickets.shape
m, n

(190, 20)

In [178]:
possibilities = {}

for index in range(n):
    possibilities[index] = list(rules.keys())

In [179]:
for ticket in valid_tickets:
    for i, value in enumerate(ticket):
        for rule, ranges in rules.items():
            if int(value) not in range(*ranges[0]) and int(value) not in range(*ranges[1]):
                possibilities[i].remove(rule)

In [196]:
finished = True

while finished:
    
    for index, possibility in possibilities.items():
        if len(possibility) == 1:
            for i, p in possibilities.items():
                if possibility[0] in p and i != index:
                    p.remove(possibility[0])
    
    finished = not all([ len(p) == 1 for p in possibilities.values() ])

In [197]:
possibilities

{0: ['arrival track'],
 1: ['duration'],
 2: ['departure time'],
 3: ['departure station'],
 4: ['class'],
 5: ['type'],
 6: ['departure date'],
 7: ['wagon'],
 8: ['arrival platform'],
 9: ['price'],
 10: ['arrival location'],
 11: ['row'],
 12: ['departure platform'],
 13: ['zone'],
 14: ['arrival station'],
 15: ['departure location'],
 16: ['train'],
 17: ['route'],
 18: ['departure track'],
 19: ['seat']}

In [204]:
my_ticket = np.array(list(map(lambda s: int(s), my_ticket)))
np.sum(np.take(my_ticket, [2, 3, 6, 12, 15, 18]))

618